In [1]:
#importing libraries and modules
from bs4 import BeautifulSoup
import requests
import numpy as np
import pandas as pd
from geopy.geocoders import Nominatim
import folium
import json
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

In [2]:
#fetching URL 
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
result = requests.get(url)
print(url)
print(result.status_code)
print(result.headers)

https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M
200
{'Date': 'Sat, 12 Jun 2021 06:55:52 GMT', 'Server': 'mw1272.eqiad.wmnet', 'X-Content-Type-Options': 'nosniff', 'P3p': 'CP="See https://en.wikipedia.org/wiki/Special:CentralAutoLogin/P3P for more info."', 'Content-Language': 'en', 'Vary': 'Accept-Encoding,Cookie,Authorization', 'Last-Modified': 'Sat, 29 May 2021 06:55:52 GMT', 'Content-Type': 'text/html; charset=UTF-8', 'Content-Encoding': 'gzip', 'Age': '15232', 'X-Cache': 'cp5008 miss, cp5008 hit/7', 'X-Cache-Status': 'hit-front', 'Server-Timing': 'cache;desc="hit-front", host;desc="cp5008"', 'Strict-Transport-Security': 'max-age=106384710; includeSubDomains; preload', 'Report-To': '{ "group": "wm_nel", "max_age": 86400, "endpoints": [{ "url": "https://intake-logging.wikimedia.org/v1/events?stream=w3c.reportingapi.network_error&schema_uri=/w3c/reportingapi/network_error/1.0.0" }] }', 'NEL': '{ "report_to": "wm_nel", "max_age": 86400, "failure_fraction": 0.05, "success

In [3]:
#web scrapping
table_contents=[]
soup = BeautifulSoup(result.content, 'html.parser')
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

In [4]:
table_contents

[{'PostalCode': 'M3A', 'Borough': 'North York', 'Neighborhood': 'Parkwoods'},
 {'PostalCode': 'M4A',
  'Borough': 'North York',
  'Neighborhood': 'Victoria Village'},
 {'PostalCode': 'M5A',
  'Borough': 'Downtown Toronto',
  'Neighborhood': 'Regent Park, Harbourfront'},
 {'PostalCode': 'M6A',
  'Borough': 'North York',
  'Neighborhood': 'Lawrence Manor, Lawrence Heights'},
 {'PostalCode': 'M7A',
  'Borough': "Queen's Park",
  'Neighborhood': 'Ontario Provincial Government'},
 {'PostalCode': 'M9A',
  'Borough': 'Etobicoke',
  'Neighborhood': 'Islington Avenue'},
 {'PostalCode': 'M1B',
  'Borough': 'Scarborough',
  'Neighborhood': 'Malvern, Rouge'},
 {'PostalCode': 'M3B',
  'Borough': 'North York',
  'Neighborhood': 'Don Mills North'},
 {'PostalCode': 'M4B',
  'Borough': 'East York',
  'Neighborhood': 'Parkview Hill, Woodbine Gardens'},
 {'PostalCode': 'M5B',
  'Borough': 'Downtown Toronto',
  'Neighborhood': 'Garden District, Ryerson'},
 {'PostalCode': 'M6B', 'Borough': 'North York', 'N

In [5]:
#converting to dataframe
df = pd.DataFrame(table_contents)

In [6]:
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


In [7]:
#grouping postalcode with borough and neighbourhood
df = df.groupby('PostalCode').agg(
    {
        'Borough':'first', 
        'Neighborhood': ', '.join,}
    ).reset_index()

In [8]:
df.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [9]:
df.loc[df['PostalCode'] == 'M5A']

,PostalCode,Borough,Neighborhood
53,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [10]:
df.shape

(103, 3)

In [11]:
dfgeo = pd.read_csv("Geospatial_Coordinates.csv")
dfgeo.rename(columns={'Postal Code': 'PostalCode'}, inplace=True)

In [12]:
df2 = pd.merge(df, dfgeo, on="PostalCode", how='left')

In [13]:
df2.loc[df2['PostalCode'] == 'M5G']

,PostalCode,Borough,Neighborhood,Latitude,Longitude
57,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383


In [14]:
df2.loc[df2['PostalCode'] == 'M9V']

,PostalCode,Borough,Neighborhood,Latitude,Longitude
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437


In [15]:
df2.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [16]:
CLIENT_ID = 'CHHMF0324VJ4S4BR445TFQLQZSTQGTZUDDLSCXXMN3MGFY4F'# Foursquare Id
CLIENT_SECRET = 'VTAHFIRGU0K3FBKFTLKRDQKUQXGFSHT3AAAPQFCGAUWLXTXF' # Foursquare Secret
VERSION = '20180605' # API version

In [17]:
df2.set_index('PostalCode', inplace = True) 
neighborhood_latitude = df2.loc['M9V']['Latitude']
neighborhood_longitude = df2.loc['M9V']['Longitude']

In [18]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=CHHMF0324VJ4S4BR445TFQLQZSTQGTZUDDLSCXXMN3MGFY4F&client_secret=VTAHFIRGU0K3FBKFTLKRDQKUQXGFSHT3AAAPQFCGAUWLXTXF&v=20180605&ll=43.7394164,-79.5884369&radius=500&limit=100'

In [19]:
results = requests.get(url).json()

In [20]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [21]:
venues = results['response']['groups'][0]['items']

In [22]:
nearby_venues = json_normalize(venues) # flatten JSON
nearby_venues

<ipython-input-22-ea6da929ed3c>:1: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  nearby_venues = json_normalize(venues) # flatten JSON


,referralId,reasons.count,reasons.items,venue.id,venue.name,venue.location.address,venue.location.crossStreet,venue.location.lat,venue.location.lng,venue.location.labeledLatLngs,venue.location.distance,venue.location.postalCode,venue.location.cc,venue.location.city,venue.location.state,venue.location.country,venue.location.formattedAddress,venue.categories,venue.photos.count,venue.photos.groups
0,e-0-4c633939e1621b8d48842553-0,0,"[{'summary': 'This spot is popular', 'type': '...",4c633939e1621b8d48842553,Subway,"6210 Finch Ave West, Store 103",at Albion Rd.,43.742645,-79.589643,"[{'label': 'display', 'lat': 43.74264512142215...",372,M9V 0A1,CA,Toronto,ON,Canada,"[6210 Finch Ave West, Store 103 (at Albion Rd....","[{'id': '4bf58dd8d48988d1c5941735', 'name': 'S...",0,[]
1,e-0-4cd4738cdfb4a1cd4337535c-1,0,"[{'summary': 'This spot is popular', 'type': '...",4cd4738cdfb4a1cd4337535c,The Beer Store,1530 Albion Rd,Near Finch Ave. W.,43.741694,-79.584373,"[{'label': 'display', 'lat': 43.7416936, 'lng'...",413,M9V 1B4,CA,Etobicoke,ON,Canada,"[1530 Albion Rd (Near Finch Ave. W.), Etobicok...","[{'id': '5370f356bcbc57f1066c94c2', 'name': 'B...",0,[]
2,e-0-4be58dc4cf200f479154133c-2,0,"[{'summary': 'This spot is popular', 'type': '...",4be58dc4cf200f479154133c,Shoppers Drug Mart,1530 Albion Rd,Albion Mall,43.741685,-79.584487,"[{'label': 'display', 'lat': 43.741685, 'lng':...",405,M9V 1B4,CA,Etobicoke,ON,Canada,"[1530 Albion Rd (Albion Mall), Etobicoke ON M9...","[{'id': '4bf58dd8d48988d10f951735', 'name': 'P...",0,[]
3,e-0-4d8ba6910c4e41bdaaf7667f-3,0,"[{'summary': 'This spot is popular', 'type': '...",4d8ba6910c4e41bdaaf7667f,Pizza Pizza,"1530 Albion Road, Unit T25",NaN,43.741569,-79.584489,"[{'label': 'display', 'lat': 43.74156896801906...",397,M9V 1B4,CA,Etobicoke,ON,Canada,"[1530 Albion Road, Unit T25, Etobicoke ON M9V ...","[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",0,[]
4,e-0-4b04a05bf964a520c45522e3-4,0,"[{'summary': 'This spot is popular', 'type': '...",4b04a05bf964a520c45522e3,Sheriff's No Frills,1530 Albion Rd,at Finch Ave. W.,43.741696,-79.584379,"[{'label': 'display', 'lat': 43.741696, 'lng':...",413,M9V 1B4,CA,Etobicoke,ON,Canada,"[1530 Albion Rd (at Finch Ave. W.), Etobicoke ...","[{'id': '4bf58dd8d48988d118951735', 'name': 'G...",0,[]
5,e-0-4c1951d6834e2d7f2d3a2a80-5,0,"[{'summary': 'This spot is popular', 'type': '...",4c1951d6834e2d7f2d3a2a80,McDonald's,"1530 Albion Road, Unit F-1",NaN,43.741757,-79.584230,"[{'label': 'display', 'lat': 43.7417571, 'lng'...",427,M9V 1B4,CA,Toronto,ON,Canada,"[1530 Albion Road, Unit F-1, Toronto ON M9V 1B...","[{'id': '4bf58dd8d48988d16e941735', 'name': 'F...",0,[]
6,e-0-5112b872e4b0c0a78d7bcd27-6,0,"[{'summary': 'This spot is popular', 'type': '...",5112b872e4b0c0a78d7bcd27,Sunny Foodmart,1620 Albion road,Albion Road and Finch Ave,43.741840,-79.590561,"[{'label': 'display', 'lat': 43.74184023292111...",319,NaN,CA,Toronto,ON,Canada,"[1620 Albion road (Albion Road and Finch Ave),...","[{'id': '4bf58dd8d48988d118951735', 'name': 'G...",0,[]
7,e-0-4cd9d00734bb8cfa6576babf-7,0,"[{'summary': 'This spot is popular', 'type': '...",4cd9d00734bb8cfa6576babf,Tim Hortons,6220 Finch Avenue West,Albion,43.742015,-79.589690,"[{'label': 'display', 'lat': 43.7420146612893,...",306,M9V 0A1,CA,Etobicoke,ON,Canada,"[6220 Finch Avenue West (Albion), Etobicoke ON...","[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",0,[]
8,e-0-4f035c5c5c5c51dd31b52e55-8,0,"[{'summary': 'This spot is popular', 'type': '...",4f035c5c5c5c51dd31b52e55,Dollarama,"1620 Albion Rd, Finch Albion Centre",NaN,43.742018,-79.591076,"[{'label': 'display', 'lat': 43.742018, 'lng':...",359,M9V 4B4,CA,Etobicoke,ON,Canada,"[1620 Albion Rd, Finch Albion Centre, Etobicok...","[{'id': '52dea92d3cf9994f4e043dbb', 'name': 'D...",0,[]
9,e-0-54e521a9498e559c968e8083-9,0,"[{'summary': 'This spot is popular', 'type': '...",54e521a9498e559c968e8083,NORI SUSHI,NaN,NaN,43.742775,-79.586985,"[{'label': 'display', 'lat': 43.742775, 'lng':...",391,N

In [23]:

filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues = nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Subway,Sandwich Place,43.742645,-79.589643
1,The Beer Store,Beer Store,43.741694,-79.584373
2,Shoppers Drug Mart,Pharmacy,43.741685,-79.584487
3,Pizza Pizza,Pizza Place,43.741569,-79.584489
4,Sheriff's No Frills,Grocery Store,43.741696,-79.584379


In [24]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

10 venues were returned by Foursquare.


In [25]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [26]:
df2

,Borough,Neighborhood,Latitude,Longitude
PostalCode,,,,
M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
M1G,Scarborough,Woburn,43.770992,-79.216917
M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...
M9N,York,Weston,43.706876,-79.518188
M9P,Etobicoke,Westmount,43.696319,-79.532242
M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",43.688905,-79.554724


In [27]:
venues = getNearbyVenues(names=df2['Borough'],latitudes=df2['Latitude'],longitudes=df2['Longitude'])

Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
Scarborough
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
North York
East York
East York
East Toronto
East York
East York
East YorkEast Toronto
East Toronto
East Toronto
East Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Central Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
North York
Central Toronto
Central Toronto
Central Toronto
Downtown Toronto
Downtown Toronto
Downtown Toronto
Downtown TorontoStn A PO Boxes25 The Esplanade
Downtown Toronto
North York

In [28]:
print(venues.shape)
venues.head()

(2012, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Scarborough,43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
1,Scarborough,43.784535,-79.160497,Great Shine Window Cleaning,43.783145,-79.157431,Home Service
2,Scarborough,43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
3,Scarborough,43.763573,-79.188711,RBC Royal Bank,43.766790,-79.191151,Bank
4,Scarborough,43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store


In [29]:
venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Central Toronto,96,96,96,96,96,96
Downtown Toronto,1070,1070,1070,1070,1070,1070
Downtown TorontoStn A PO Boxes25 The Esplanade,85,85,85,85,85,85
East Toronto,93,93,93,93,93,93
East TorontoBusiness reply mail Processing Centre969 Eastern,17,17,17,17,17,17
East York,63,63,63,63,63,63
East YorkEast Toronto,3,3,3,3,3,3
Etobicoke,67,67,67,67,67,67
EtobicokeNorthwest,4,4,4,4,4,4


In [30]:
print('There are {} unique categories.'.format(len(venues['Venue Category'].unique())))

There are 256 unique categories.


In [31]:
#one hot encoding
onehot = pd.get_dummies(venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
onehot['Neighborhood'] = venues['Neighborhood'] 

onehot.head()

,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [32]:
onehot.shape

(2012, 256)

In [33]:
grouped = onehot.groupby('Neighborhood').mean().reset_index()
grouped

,Neighborhood,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Central Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010417,...,0.000000,0.000000,0.000000,0.010417,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,Downtown Toronto,0.000935,0.000935,0.000935,0.000935,0.000935,0.001869,0.002804,0.001869,0.009346,...,0.000000,0.011215,0.001869,0.002804,0.000000,0.007477,0.001869,0.000000,0.000000,0.002804
2,Downtown TorontoStn A PO Boxes25 The Esplanade,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.023529,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,East Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.021505,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010753
4,East TorontoBusiness reply mail Processing Cen...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.058824
5,East York,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.015873,0.000000,0.000000,0.000000,0.000000,0.000000
6,East YorkEast Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,Etobicoke,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.014925,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.014925,0.000000,0.000000
8,EtobicokeNorthwest,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,MississaugaCanada Post Gateway Processing Centre,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.071429,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [34]:
grouped.shape

(15, 256)

In [35]:
#Getting the 5 most visited venues in the different neighborhoods
num_top_venues = 5

for hood in grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = grouped[grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Central Toronto----
            venue  freq
0     Coffee Shop  0.09
1            Park  0.07
2  Sandwich Place  0.06
3     Pizza Place  0.06
4            Café  0.04


----Downtown Toronto----
                 venue  freq
0          Coffee Shop  0.12
1                 Café  0.06
2       Sandwich Place  0.05
3  Japanese Restaurant  0.03
4           Restaurant  0.03


----Downtown TorontoStn A PO Boxes25 The Esplanade----
                venue  freq
0      Sandwich Place  0.07
1         Coffee Shop  0.06
2  Italian Restaurant  0.05
3        Cocktail Bar  0.05
4              Bakery  0.04


----East Toronto----
                venue  freq
0    Greek Restaurant  0.11
1         Coffee Shop  0.08
2  Italian Restaurant  0.05
3                Park  0.04
4      Ice Cream Shop  0.03


----East TorontoBusiness reply mail Processing Centre969 Eastern----
                venue  freq
0  Light Rail Station  0.12
1         Yoga Studio  0.06
2       Auto Workshop  0.06
3          Comic Shop  0.06
4   

In [36]:
#Getting the dataframe for 10 most visited venues in the different neighborhoods
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [37]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = grouped['Neighborhood']

for ind in np.arange(grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central Toronto,Coffee Shop,Park,Sandwich Place,Pizza Place,Café,Gym,Sushi Restaurant,Clothing Store,Thai Restaurant,Sporting Goods Shop
1,Downtown Toronto,Coffee Shop,Café,Sandwich Place,Japanese Restaurant,Restaurant,Hotel,Sushi Restaurant,Bank,Gym,Italian Restaurant
2,Downtown TorontoStn A PO Boxes25 The Esplanade,Sandwich Place,Coffee Shop,Italian Restaurant,Cocktail Bar,Bakery,Japanese Restaurant,Bank,Hotel,Café,Beer Bar
3,East Toronto,Greek Restaurant,Coffee Shop,Italian Restaurant,Park,Ice Cream Shop,Café,Sandwich Place,Pub,Pizza Place,Bakery
4,East TorontoBusiness reply mail Processing Cen...,Light Rail Station,Yoga Studio,Auto Workshop,Comic Shop,Recording Studio,Restaurant,Butcher,Skate Park,Burrito Place,Farmers Market
5,East York,Coffee Shop,Bank,Sporting Goods Shop,Intersection,Pizza Place,Burger Joint,Sandwich Place,Park,Athletics & Sports,Supermarket
6,East YorkEast Toronto,Intersection,Park,Convenience Store,Adult Boutique,Miscellaneous Shop,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop,Mexican Restaurant
7,Etobicoke,Pizza Place,Coffee Shop,Sandwich Place,Pharmacy,Discount Store,Grocery Store,Café,Middle Eastern Restaurant,Gym,Park
8,EtobicokeNorthwest,Drugstore,Truck Stop,Rental Car Location,Garden Center,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Moroccan Restaurant,Miscellaneous Shop
9,MississaugaCanada Post Gateway Processing Centre,Coffee Shop,Hotel,American Restaurant,Sandwich Place,Mediterranean Restaurant,Fried Chicken Joint,Intersection,Gym,Gas Station,Burrito Place


In [38]:
# set number of clusters
kclusters = 5

grouped_clustering = grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 0, 3, 0, 2, 0, 1, 4])

In [39]:
# add clustering labels
merged=grouped
merged['Cluster_Labels'] = kmeans.labels_



# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
merged = merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

merged.head() # check the last columns!

,Neighborhood,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010417,...,Coffee Shop,Park,Sandwich Place,Pizza Place,Café,Gym,Sushi Restaurant,Clothing Store,Thai Restaurant,Sporting Goods Shop
1,Downtown Toronto,0.000935,0.000935,0.000935,0.000935,0.000935,0.001869,0.002804,0.001869,0.009346,...,Coffee Shop,Café,Sandwich Place,Japanese Restaurant,Restaurant,Hotel,Sushi Restaurant,Bank,Gym,Italian Restaurant
2,Downtown TorontoStn A PO Boxes25 The Esplanade,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,Sandwich Place,Coffee Shop,Italian Restaurant,Cocktail Bar,Bakery,Japanese Restaurant,Bank,Hotel,Café,Beer Bar
3,East Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.021505,...,Greek Restaurant,Coffee Shop,Italian Restaurant,Park,Ice Cream Shop,Café,Sandwich Place,Pub,Pizza Place,Bakery
4,East TorontoBusiness reply mail Processing Cen...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,Light Rail Station,Yoga Studio,Auto Workshop,Comic Shop,Recording Studio,Restaurant,Butcher,Skate Park,Burrito Place,Farmers Market


In [40]:
merged=merged.dropna()

In [41]:
merged.head()

,Neighborhood,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010417,...,Coffee Shop,Park,Sandwich Place,Pizza Place,Café,Gym,Sushi Restaurant,Clothing Store,Thai Restaurant,Sporting Goods Shop
1,Downtown Toronto,0.000935,0.000935,0.000935,0.000935,0.000935,0.001869,0.002804,0.001869,0.009346,...,Coffee Shop,Café,Sandwich Place,Japanese Restaurant,Restaurant,Hotel,Sushi Restaurant,Bank,Gym,Italian Restaurant
2,Downtown TorontoStn A PO Boxes25 The Esplanade,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,Sandwich Place,Coffee Shop,Italian Restaurant,Cocktail Bar,Bakery,Japanese Restaurant,Bank,Hotel,Café,Beer Bar
3,East Toronto,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.021505,...,Greek Restaurant,Coffee Shop,Italian Restaurant,Park,Ice Cream Shop,Café,Sandwich Place,Pub,Pizza Place,Bakery
4,East TorontoBusiness reply mail Processing Cen...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,Light Rail Station,Yoga Studio,Auto Workshop,Comic Shop,Recording Studio,Restaurant,Butcher,Skate Park,Burrito Place,Farmers Market


In [42]:
merged['Cluster_Labels']

0     0
1     0
2     0
3     0
4     3
5     0
6     2
7     0
8     1
9     4
10    0
11    0
12    0
13    0
14    0
Name: Cluster_Labels, dtype: int32

In [43]:
merged['Cluster_Labels'] = merged['Cluster_Labels'].astype(int)

In [44]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="Toronto")
location = geolocator.geocode(address)
latitude_toronto = location.latitude
longitude_toronto = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude_toronto, longitude_toronto))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [45]:
map_toronto = folium.Map(location=[latitude_toronto, longitude_toronto], zoom_start=10)

# add markers to map
for lat, lng, borough, Neighborhood in zip(df2['Latitude'], df2['Longitude'], df2['Borough'], df2['Neighborhood']):
    label = '{}, {}'.format(Neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Examine Clusters

In [46]:
#cluster 1
merged.loc[merged['Cluster_Labels'] == 0,merged.columns[[1] + list(range(5,merged.shape[1]))]]

,Adult Boutique,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,...,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.010417,0.000000,0.000000,0.000000,0.000000,...,Coffee Shop,Park,Sandwich Place,Pizza Place,Café,Gym,Sushi Restaurant,Clothing Store,Thai Restaurant,Sporting Goods Shop
1,0.000935,0.000935,0.001869,0.002804,0.001869,0.009346,0.000935,0.002804,0.007477,0.001869,...,Coffee Shop,Café,Sandwich Place,Japanese Restaurant,Restaurant,Hotel,Sushi Restaurant,Bank,Gym,Italian Restaurant
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.011765,0.000000,0.023529,0.000000,...,Sandwich Place,Coffee Shop,Italian Restaurant,Cocktail Bar,Bakery,Japanese Restaurant,Bank,Hotel,Café,Beer Bar
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.021505,0.000000,0.000000,0.000000,0.010753,...,Greek Restaurant,Coffee Shop,Italian Restaurant,Park,Ice Cream Shop,Café,Sandwich Place,Pub,Pizza Place,Bakery
5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,Coffee Shop,Bank,Sporting Goods Shop,Intersection,Pizza Place,Burger Joint,Sandwich Place,Park,Athletics & Sports,Supermarket
7,0.000000,0.000000,0.000000,0.000000,0.000000,0.014925,0.000000,0.000000,0.000000,0.000000,...,Pizza Place,Coffee Shop,Sandwich Place,Pharmacy,Discount Store,Grocery Store,Café,Middle Eastern Restaurant,Gym,Park
10,0.000000,0.000000,0.000000,0.000000,0.000000,0.009132,0.000000,0.000000,0.004566,0.004566,...,Coffee Shop,Clothing Store,Park,Grocery Store,Japanese Restaurant,Bank,Restaurant,Fast Food Restaurant,Sandwich Place,Pizza Place
11,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,Coffee Shop,Sushi Restaurant,Burrito Place,Yoga Studio,Japanese Restaurant,Burger Joint,Café,Sandwich Place,Salad Place,Bar
12,0.000000,0.000000,0.000000,0.000000,0.000000,0.011364,0.000000,0.000000,0.000000,0.000000,...,Fast Food Restaurant,Coffee Shop,Chinese Restaurant,Bank,Breakfast Spot,Pizza Place,Bakery,Middle Eastern Restaurant,Skating Rink,Sandwich Place
13,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.006803,0.000000,...,Coffee Shop,Café,Bar,Italian Restaurant,Grocery Store,Bakery,Restaurant,Yoga Studio,Sushi Restaurant,Bank


In [47]:
#cluster 2
merged.loc[merged['Cluster_Labels'] == 1,merged.columns[[1] + list(range(5,merged.shape[1]))]]

,Adult Boutique,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,...,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,Drugstore,Truck Stop,Rental Car Location,Garden Center,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Moroccan Restaurant,Miscellaneous Shop


In [48]:
#cluster 3
merged.loc[merged['Cluster_Labels'] == 2,merged.columns[[1] + list(range(5,merged.shape[1]))]]

,Adult Boutique,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,...,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,Intersection,Park,Convenience Store,Adult Boutique,Miscellaneous Shop,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop,Mexican Restaurant


In [49]:
#cluster 4
merged.loc[merged['Cluster_Labels'] == 3,merged.columns[[1] + list(range(5,merged.shape[1]))]]

,Adult Boutique,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,...,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,Light Rail Station,Yoga Studio,Auto Workshop,Comic Shop,Recording Studio,Restaurant,Butcher,Skate Park,Burrito Place,Farmers Market
